In [1]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm
import os


# data_dir = Path('/home/jovyan/nas/1_user/ghdud1519@agilesoda.ai/ai4code/rawdata')
# if not os.path.exists("./data"):
#     os.mkdir("./data")

data_dir = Path('./input/AI4Code')

def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
            .assign(id=path.stem)
            .rename_axis('cell_id')
    )


## Data import & Time
- Total 
    - 1H 5Min
- Train: 20,000 / Valid: 10,000
    - 5Min

In [2]:
paths_train = list((data_dir / 'train').glob('*.json'))
# paths_train = paths_train[:30000]
print(len(paths_train))

notebooks_train = [
    read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
]
df = (
    pd.concat(notebooks_train)
        .set_index('id', append=True)
        .swaplevel()
        .sort_index(level='id', sort_remaining=False)
)

139256


Train NBs: 100%|███████████████████████| 139256/139256 [07:14<00:00, 320.62it/s]


In [3]:
#
print(len(paths_train))

139256


In [5]:
df.head(5)

cell_type  \
id             cell_id              
00001756c60be8 1862f0a6      code   
               2a9e43d6      code   
               038b763d      code   
               2eefe0ef      code   
               0beab1cd      code   

                                                                    source  
id             cell_id                                                      
00001756c60be8 1862f0a6  # This Python 3 environment comes with many he...  
               2a9e43d6  import numpy as np\nimport pandas as pd\nimpor...  
               038b763d  import warnings\nwarnings.filterwarnings('igno...  
               2eefe0ef      matplotlib.rcParams.update({'font.size': 14})  
               0beab1cd  def evaluate_preds(train_true_values, train_pr...

- Train / Valid Data Sample

In [8]:
#
# notebook id lst
# set 처리
nb_id_lst = list(set(df.index.get_level_values('id').tolist()))
len(nb_id_lst)

139256

In [9]:
nb_id_lst[:10]

['1ffa22d6813f58',
 'a4c69bee78a0db',
 '1c92c3e5d9a67c',
 'a3811c681a66fb',
 'ec727961f92e82',
 'c3af3a287bb026',
 '0dd359a7df0c38',
 '62b72c902b7e24',
 '763c921d10da8f',
 '3ade9fc6c07930']

In [11]:
#
# Data sampling
import random

random.seed(1234)
# random.sample() # 인자가 주어지지 않아 TypeError

# random.sample(sequence, k)
# sequence에서 지정한 숫자만큼의 요소들을 랜덤으로 뽑아 리스트로 반환

In [12]:
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()  # Split the string representation of cell_ids into a list

/tmp/ipykernel_96777/1429457819.py:1: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df_orders = pd.read_csv(


In [13]:
print(df_orders.shape)
df_orders.head(5)

(139256,)


id
00001756c60be8    [1862f0a6, 448eb224, 2a9e43d6, 7e2f170a, 038b7...
00015c83e2717b    [2e94bd7a, 3e99dee9, b5e286ea, da4f7550, c4172...
0001bdd4021779    [3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310...
0001daf4c2c76d    [97266564, a898e555, 86605076, 76cc2642, ef279...
0002115f48f982    [9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe...
Name: cell_order, dtype: object

In [14]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]


df_orders_ = df_orders.to_frame().join(
    df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
    how='right',
)

In [15]:
df_orders_

,cell_order,cell_id
id,,
00001756c60be8,"[1862f0a6, 448eb224, 2a9e43d6, 7e2f170a, 038b7...","[1862f0a6, 2a9e43d6, 038b763d, 2eefe0ef, 0beab..."
00015c83e2717b,"[2e94bd7a, 3e99dee9, b5e286ea, da4f7550, c4172...","[c417225b, 51e3cd89, 2600b4eb, 75b65993, cf195..."
0001bdd4021779,"[3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310...","[3fdc37be, 073782ca, 8ea7263c, 80543cd8, 38310..."
0001daf4c2c76d,"[97266564, a898e555, 86605076, 76cc2642, ef279...","[86605076, df6c939f, 00f87d0a, 986fd4f1, 9b761..."
0002115f48f982,"[9ec225f0, 18281c6c, e3b6b115, 4a044c54, 365fe...","[18281c6c, e3b6b115, 4a044c54, 365fe576, a3188..."
...,...,...
fffc30d5a0bc46,"[09727c0c, ff1ea6a0, ddfef603, a01ce9b3, 3ba95...","[ff1ea6a0, a01ce9b3, bf92a015, 095812e6, aa32a..."
fffc3b44869198,"[978a5137, faa48f03, 28dfb12a, eea2e812, 64fef...","[978a5137, faa48f03, 28dfb12a, eea2e812, 64fef..."
fffc63ff750064,"[5015c300, 411b85d9, 8238198c, f4781d1d, b5532...","[5015c300, 8238198c, f4781d1d, b5532930, e1f22..."


In [16]:
ranks = {}
for id_, cell_order, cell_id in df_orders_.itertuples():
    ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}

df_ranks = (
    pd.DataFrame
        .from_dict(ranks, orient='index')
        .rename_axis('id')
        .apply(pd.Series.explode)
        .set_index('cell_id', append=True)
)

In [17]:
df_ranks

rank
id             cell_id      
00001756c60be8 1862f0a6    0
               2a9e43d6    2
               038b763d    4
               2eefe0ef    6
               0beab1cd    8
...                      ...
fffe1d764579d5 0d770d6b   43
               d45ddc62   33
               1a63248d    0
               a8ffc8b4   69
               4e2d4c2d    3

[6370646 rows x 1 columns]

In [19]:
df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')
df = df.reset_index().merge(df_ranks, on=["id", "cell_id"]).merge(df_ancestors, on=["id"])
df["pct_rank"] = df["rank"] / df.groupby("id")["cell_id"].transform("count")

In [20]:
df_ancestors.tail(5)

,ancestor_id,parent_id
id,,
fffc30d5a0bc46,6aed207b,NaN
fffc3b44869198,a6aaa8d7,NaN
fffc63ff750064,0a1b5b65,NaN
fffcd063cda949,d971e960,NaN
fffe1d764579d5,3c40bfa6,NaN


In [21]:
df.tail(5)

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
6370641,fffe1d764579d5,0d770d6b,markdown,## REMOVING THE OUTLIERS,43,3c40bfa6,NaN,0.597222
6370642,fffe1d764579d5,d45ddc62,markdown,### DIMENSIONALITY CURSE,33,3c40bfa6,NaN,0.458333
6370643,fffe1d764579d5,1a63248d,markdown,# BANGALORE HOUSE PRICE PREDICTION,0,3c40bfa6,NaN,0.0
6370644,fffe1d764579d5,a8ffc8b4,markdown,* We have achieved 75.2% accuracy in predictin...,69,3c40bfa6,NaN,0.958333
6370645,fffe1d764579d5,4e2d4c2d,markdown,## DATA INGESTION,3,3c40bfa6,NaN,0.041667


## GroupShuffle
### Ref : https://davinci-ai.tistory.com/18

In [22]:
from sklearn.model_selection import GroupShuffleSplit

NVALID = 0.001  # size of validation set

splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=4321)
train_ind, val_ind = next(splitter.split(df, groups=df["ancestor_id"]))

train_df = df.loc[train_ind].reset_index(drop=True)
val_df = df.loc[val_ind].reset_index(drop=True)

In [23]:
# print(train_df.shape, val_df.shape)

In [24]:
print(train_df.shape, val_df.shape)

(6364812, 8) (5834, 8)


In [25]:
print(train_ind[0:10])

[0 1 2 3 4 5 6 7 8 9]


In [26]:
# save_path = '/home/jovyan/nas/1_user/ghdud1519@agilesoda.ai/ai4code/ai4code-preprocess-999/'
save_path = './ai4code-preprocess-999/'


In [27]:
# Base markdown dataframes
train_df_mark = train_df[train_df["cell_type"] == "markdown"].reset_index(drop=True)
val_df_mark = val_df[val_df["cell_type"] == "markdown"].reset_index(drop=True)

train_df_mark.to_csv(save_path + "train_mark.csv", index=False)
val_df_mark.to_csv(save_path + "val_mark.csv", index=False)
val_df.to_csv(save_path + "val.csv", index=False)
train_df.to_csv(save_path + "train.csv", index=False)

In [28]:
train_df_mark.tail(2)

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
2164085,fffe1d764579d5,a8ffc8b4,markdown,* We have achieved 75.2% accuracy in predictin...,69,3c40bfa6,NaN,0.958333
2164086,fffe1d764579d5,4e2d4c2d,markdown,## DATA INGESTION,3,3c40bfa6,NaN,0.041667


In [29]:
val_df.tail(2)

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
5832,ff4e1afc534c6f,70ee3cd3,markdown,# **XGBOOST**,79,a7923311,NaN,0.908046
5833,ff4e1afc534c6f,e5018708,markdown,# **Introduction**,0,a7923311,NaN,0.0


In [30]:
val_df_mark.tail(2)

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
1975,ff4e1afc534c6f,70ee3cd3,markdown,# **XGBOOST**,79,a7923311,NaN,0.908046
1976,ff4e1afc534c6f,e5018708,markdown,# **Introduction**,0,a7923311,NaN,0.0


In [31]:
val_df[val_df["cell_type"] != "markdown"].tail(2)

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
5805,ff4e1afc534c6f,47a7075f,code,\n#submission = pd.read_csv('submission1.csv')...,83,a7923311,NaN,0.954023
5806,ff4e1afc534c6f,f6664471,code,clf = DecisionTreeClassifier()\nscoring = 'acc...,85,a7923311,NaN,0.977011


## Additional Code cells


In [32]:
# Additional code cells
def clean_code(cell):
    return str(cell).replace("\\n", "\n")


def sample_cells(cells, n):
    cells = [clean_code(cell) for cell in cells]
    if n >= len(cells):
        return [cell[:200] for cell in cells]
    else:
        results = []
        step = len(cells) / n
        idx = 0
        while int(np.round(idx)) < len(cells):
            results.append(cells[int(np.round(idx))])
            idx += step
        assert cells[0] in results
        if cells[-1] not in results:
            results[-1] = cells[-1]
        return results


def get_features(df):
    features = dict()
    df = df.sort_values("rank").reset_index(drop=True)
    for idx, sub_df in tqdm(df.groupby("id")):
        features[idx] = dict()
        total_md = sub_df[sub_df.cell_type == "markdown"].shape[0]
        code_sub_df = sub_df[sub_df.cell_type == "code"]
        total_code = code_sub_df.shape[0]
        codes = sample_cells(code_sub_df.source.values, 20)
        features[idx]["total_code"] = total_code
        features[idx]["total_md"] = total_md
        features[idx]["codes"] = codes
    return features

In [33]:
val_fts = get_features(val_df)
json.dump(val_fts, open(save_path + "val_fts.json","wt"))
train_fts = get_features(train_df)
json.dump(train_fts, open(save_path + "train_fts.json","wt"))

100%|█████████████████████████████████| 139112/139112 [01:33<00:00, 1487.17it/s]


In [35]:
val_df.iloc[10]['source']

'shops.shape'

In [36]:
len(val_df['source'].str.contains('\#'))

5834

In [37]:
val_df['source']

0       # This Python 3 environment comes with many he...
1       import numpy as np\nimport matplotlib.pyplot a...
2       train=pd.read_csv("../input/competitive-data-s...
3                                             train.shape
4                                            train.head()
                              ...                        
5829                                    # **Naive Beyes**
5830                                         # **Gender**
5831                                            # **Age**
5832                                        # **XGBOOST**
5833                                   # **Introduction**
Name: source, Length: 5834, dtype: object

In [38]:
# 주석이 안들어간 셀은 없는 듯
len(val_df['source'].str.contains('#'))/len(val_df)

1.0

In [39]:
from collections import deque

def preprocess_script(script):
    new_script = deque()
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
                continue
            line = line.rstrip()
            if '#' in line:
                line = line[:line.index('#')] # 주석 전까지 코드만 저장
            line = line.replace('\n','') # 개행 문자를 모두 삭제함
            line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
            
            if line == '': # 전처리 후 빈 라인은 skip
                continue
            
            new_script.append(line)
            
        new_script = '\n'.join(new_script) # 개행 문자로 합침
        new_script = re.sub('("""[\w\W]*?""")', '<str>', new_script)
        new_script = re.sub("('''[\w\W]*?''')", '<str>', new_script)
        new_script = re.sub('/^(http?|https?):\/\/([a-z0-9-]+\.)+[a-z0-9]{2,4}.*$/', '', new_script)
    
    return new_script

In [40]:
val_df.iloc[10]['source']

'shops.shape'

In [44]:
val_df[:10]

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank
0,0b140e51df41a2,2c853a43,code,# This Python 3 environment comes with many he...,0,c8e84f06,NaN,0.0
1,0b140e51df41a2,dc1382e0,code,import numpy as np\nimport matplotlib.pyplot a...,2,c8e84f06,NaN,0.04
2,0b140e51df41a2,2cda6b41,code,"train=pd.read_csv(""../input/competitive-data-s...",4,c8e84f06,NaN,0.08
3,0b140e51df41a2,79aeec70,code,train.shape,6,c8e84f06,NaN,0.12
4,0b140e51df41a2,e2028ad1,code,train.head(),7,c8e84f06,NaN,0.14
5,0b140e51df41a2,d2e5aa64,code,train.info(),8,c8e84f06,NaN,0.16
6,0b140e51df41a2,09e61c4e,code,items.shape,9,c8e84f06,NaN,0.18
7,0b140e51df41a2,3210f97e,code,items.head(),10,c8e84f06,NaN,0.2
8,0b140e51df41a2,447e5654,code,categories.shape,11,c8e84f06,NaN,0.22
9,0b140e51df41a2,c27496ba,code,categories.head(),12,c8e84f06,NaN,0.24


In [43]:
val_df['cell_type'].tolist()[:10]

['code',
 'code',
 'code',
 'code',
 'code',
 'code',
 'code',
 'code',
 'code',
 'code']

In [41]:
val_df['source'].tolist()[:10]

['# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session',
 'import numpy as np\nimport matplotlib.pyplo

In [ ]:
preprocess_script(val_df['source'].tolist())

TypeError: expected str, bytes or os.PathLike object, not list

In [34]:
print(len(train_fts))
print(len(val_fts))


139112
144
